In [ ]:
import cv2
import glob
import ntpath
import numpy as np
import skimage
from skimage import morphology

In [ ]:
make_masked_biopsys()

In [ ]:
def make_masked_biopsys():

    for grade_no in range(0, 5):
        
        src_path = "G:/LiverCancer/original/" + "grade" + str(grade_no) + "/"
        src_files = glob.glob(src_path + "*.jpg")
    
        for src_file in src_files:

            print(src_file)
            
            make_masked_biopsy(src_file)

def make_masked_biopsy(src_file):

    dst_path = "G:/LiverCancer/masked_biopsy/"
    
    if "grade0" in src_file: sample_label = 0
    if "grade1" in src_file: sample_label = 1
    if "grade2" in src_file: sample_label = 2
    if "grade3" in src_file: sample_label = 3
    if "grade4" in src_file: sample_label = 4
        
   
    sample_image = cv2.imread(src_file)
    image_red = sample_image[:,:,2]
    kernel_1 = skimage.morphology.disk(20)
    kernel_2 = skimage.morphology.disk(3)

    image_closing = skimage.morphology.closing(image_red, kernel_2)
    image_erosion = skimage.morphology.erosion(image_closing, kernel_1)
    image_reconstruction_1 = skimage.morphology.reconstruction(image_erosion,image_closing).astype(np.uint8)
    image_dialtion = skimage.morphology.dilation(image_reconstruction_1, kernel_1)
    image_reconstruction_2 = skimage.morphology.reconstruction(image_dialtion, image_reconstruction_1, method='erosion').astype(np.uint8)

    image_zero = (image_reconstruction_2 - image_reconstruction_2.mean()).astype(np.int)
    image_binary = image_zero < - 10

    sample_image[image_binary] = 255
                             
    cv2.imwrite(dst_path + "grade" + str(sample_label) + "/" + ntpath.basename(src_file), sample_image)
        